In [19]:
from torchvision import models
import torch.nn as nn 
import torch
import torchvision.transforms as trn
from pathlib import Path
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from PIL import Image
import numpy as np

In [46]:
num_classes = 4

device = "cuda" 

model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1, bias=False)
model.maxpool = nn.Identity()
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

checkpoint_path = "../models/resnet50_5.pth"
checkpoint = torch.load(checkpoint_path, map_location=device)

print(checkpoint.keys())

model.load_state_dict(checkpoint)
model.eval()

#summary(model,input_size = (3, 64, 64))

odict_keys(['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.num_batches_tracked', 'layer1.0.conv2.weight', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.num_batches_tracked', 'layer1.0.conv3.weight', 'layer1.0.bn3.weight', 'layer1.0.bn3.bias', 'layer1.0.bn3.running_mean', 'layer1.0.bn3.running_var', 'layer1.0.bn3.num_batches_tracked', 'layer1.0.downsample.0.weight', 'layer1.0.downsample.1.weight', 'layer1.0.downsample.1.bias', 'layer1.0.downsample.1.running_mean', 'layer1.0.downsample.1.running_var', 'layer1.0.downsample.1.num_batches_tracked', 'layer1.1.conv1.weight', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.bn1.num_batches_tracked', 'layer1.1.conv2.we

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentu

In [47]:
IMAGES_PATH = "../data/raw/train"

# from competition page
transform = trn.Compose([
        trn.Resize((64, 64)),
        trn.ToTensor(),
        trn.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [48]:
class ImageFolderNoSubdirs(Dataset):
    def __init__(self, folder_path :Path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.image_paths: list[Path] = list(Path(folder_path).glob('*.jpeg'))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path)
        image = self.transform(image)
        return image, int(image_path.stem.split("_")[1])

dataset = ImageFolderNoSubdirs(IMAGES_PATH, transform=transform)
loader = DataLoader(dataset, batch_size=16, shuffle=False)

In [49]:
all_outputs = []
all_labels = []

triples = []

with torch.no_grad():
    for batch_images, labels in tqdm(loader, desc="Clean dataset"):
        triples.append(batch_images.view(-1,3))
        batch_images = batch_images.to(device)
        outputs = model(batch_images)
        all_outputs.append(outputs.cpu())
        all_labels.append(labels)

outputs = torch.cat(all_outputs)
labels = torch.cat(all_labels)

print(torch.std_mean(torch.cat(triples),dim=0))

Clean dataset: 100%|██████████| 83/83 [00:01<00:00, 75.86it/s]

(tensor([1.1593, 1.1594, 1.1594]), tensor([-0.0239, -0.0239, -0.0238]))


In [50]:
predicted_classes = torch.argmax(outputs,dim=1)

print(labels,predicted_classes)

accuracy = (labels == predicted_classes).float().mean().item()
print(f"Accuracy: {accuracy * 100:.2f}%")

tensor([1, 1, 1,  ..., 3, 0, 3]) tensor([1, 1, 1,  ..., 0, 0, 3])
Accuracy: 89.45%


In [12]:
# input random images, check outputs. Maybe backdoored class appears the most often?
from collections import Counter
maxes = []

checkpoint_path = "../models/resnet50_5.pth"
checkpoint = torch.load(checkpoint_path, map_location=device)

model.load_state_dict(checkpoint)

for i in range(100):
    rand_input = torch.rand((1,3,64,64),device=device)
    out = model(rand_input)
    maxes.append(torch.argmax(out).item())

print(Counter(maxes))

Counter({3: 100})
